In [24]:
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, TimeoutError, LocalCluster
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt

#code found on stack exchange
try:
    client = Client('tcp://localhost:8787', timeout='2s')
except OSError or TimeoutError:
    cluster = LocalCluster(scheduler_port=8787, n_workers=6, ip='localhost')
    client = Client(cluster)
#

client


<Client: 'tcp://127.0.0.1:8787' processes=6 threads=12, memory=16.00 GiB>

In [25]:
tweetdf = dd.read_csv(urlpath='sample_mental_health_tweets.csv', sep='\t' )

countdf = dd.read_csv(urlpath='sample_mental_health_tweet_count.csv', sep='\t')

print(tweetdf)

print(countdf)

Dask DataFrame Structure:
                tweet    time
npartitions=1                
               object  object
                  ...     ...
Dask Name: read-csv, 1 graph layer
Dask DataFrame Structure:
              timestamp  count
npartitions=1                 
                 object  int64
                    ...    ...
Dask Name: read-csv, 1 graph layer


In [26]:

tweetdf['time'] = dd.to_datetime(tweetdf['time'])

print(tweetdf.count().compute())

countdf['timestamp'] = dd.to_datetime(countdf['timestamp'])

print(countdf.count().compute())

2022-11-12 13:18:42,759 - distributed.worker - WARNING - Compute Failed
Key:       ('dataframe-count-chunk-8f9b5cabbcf26457ab6e380e8849d7e6-1c06941fc09985f3f37326983cbf2af1', 0)
Function:  execute_task
args:      ((subgraph_callable-19a5278b-55f2-43db-8095-a05ebdf08ffd, [(<function read_block_from_file at 0x115a15480>, <OpenFile '/Users/derek/Library/CloudStorage/GoogleDrive-derekdattero@gmail.com/My Drive/NYU Tandon Masters/Fall 2022/Big Data/Assignments/BigDataFinalProject/sample_mental_health_tweets.csv'>, 0, 5057179, b'\n'), None, True, True]))
kwargs:    {}
Exception: "ParserError('Error tokenizing data. C error: Expected 2 fields in line 25990, saw 3\\n')"



ParserError: Error tokenizing data. C error: Expected 2 fields in line 25990, saw 3


In [ ]:
#countdf['hour'] = countdf['timestamp'].apply    ()


#counts, bins = da.histogram(countdf, bins=len(countdf), range=[0, countdf['count'].max()])

#plt.stairs(counts, bins)